In [1]:
import os
import shutil
import logging
from dir_ops import Move
import pandas as pd
from collections import defaultdict
from SearchAlgo import DocumentSearch
from transformers import AutoTokenizer,TFBertModel #importing the tokenizer and bert model
#import keras
import tensorflow as tf
from keras.models import load_model
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')


#model = load_model("D:/Backup/Desktop/programs/NLP-text-doc-classification/customized_transformer_model.h5",custom_objects={"TFBertModel":TFBertModel})

'''#validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)
    #ssvae the value ina  variable
    dict1 = {key:value}
'''
 


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

"#validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100\nfor key , value in zip(encoded_dict.keys(),validation[0]):\n    print(key,value)\n    #ssvae the value ina  variable\n    dict1 = {key:value}\n"

In [ ]:
ex_file = r"D:\Backup\Desktop\programs\NLP-text-doc-classification\demo\reseach doc.txt"

#extract text from file and store in a variable
with open(ex_file, 'r') as f:
    text = f.read()
print(text)
encoded_dict = {
    'News': 0,
    'Research Paper': 1,
    'Code': 2,
    'Medical': 3,
    'Legal': 4,
    'Financial documents': 5
}

x_val = tokenizer(
    text=text,
    add_special_tokens=True,
    max_length=300,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

#validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
#validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
#for key , value in zip(encoded_dict.keys(),validation[0]):
   # print(key,value)
    #save the value ina  variable
   # dict1 = {key:value}

In [ ]:
#select the key with the highest value
#max_key = max(dict1, key=dict1.get)
max_key = "Research Paper"
mover = Move()
mover.classify_document(ex_file, max_key)

In [ ]:
searcher = DocumentSearch()

user_keyword = input("comma separated keywords: ").split(",")
results = searcher.retrieve_documents(user_keyword)

In [ ]:
print("Retrieved Documents:")
for i, doc in enumerate(results, start=1,):
    print(f"{i}. Title: {doc['Title']}, Category: {doc['main_category']}")# Date: {doc['date']}, Description: x")

